In [1]:
import torch 
import pandas as pd 
import torch.nn as nn 
from torch.utils.data import random_split, DataLoader, TensorDataset 
import torch.nn.functional as F 
import numpy as np 
import torch.optim as optim 
from torch.optim import Adam

In [2]:
# 加载数据
df = pd.read_excel('data\Iris_dataset.xlsx') 
print('查看数据集的样本:') 
print(df.head()) 

# 验证数据是否平衡，以及我们有哪些类型的物种  
print('\n我们的数据集是平衡的，并且有以下值需要预测:') 
print(df['Iris_Type'].value_counts())

查看数据集的样本:
     #  sepal length in cm  sepal width in cm  petal length in cm  \
0    1                 5.1                3.5                 1.4   
1  101                 6.3                3.3                 6.0   
2  110                 7.2                3.6                 6.1   
3  145                 6.7                3.3                 5.7   
4  115                 5.8                2.8                 5.1   

   petal width in cm       Iris_Type  
0                0.2     Iris-setosa  
1                2.5  Iris-virginica  
2                2.5  Iris-virginica  
3                2.5  Iris-virginica  
4                2.4  Iris-virginica  

我们的数据集是平衡的，并且有以下值需要预测:
Iris_Type
Iris-setosa        50
Iris-virginica     50
Iris-versicolor    50
Name: count, dtype: int64


In [3]:
# 将鸢尾花物种转换为数字类型：Iris-setosa=0，Iris-versicolor=1，Iris-virginica=2
labels = {'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2} 
df['IrisType_num'] = df['Iris_Type']   # 创建一个新列"IrisType_num"
df.IrisType_num = [labels[item] for item in df.IrisType_num]  # 将值转换为数字

# 定义输入和输出数据集
input = df.iloc[:, 1:-2]            # 我们丢弃第一列和最后两列。
print('\n输入值为:') 
print(input.head())   
output = df.loc[:, 'IrisType_num']   # 输出Y是最后一列  
print('\n输出值为:') 
print(output.head())


输入值为:
   sepal length in cm  sepal width in cm  petal length in cm  \
0                 5.1                3.5                 1.4   
1                 6.3                3.3                 6.0   
2                 7.2                3.6                 6.1   
3                 6.7                3.3                 5.7   
4                 5.8                2.8                 5.1   

   petal width in cm  
0                0.2  
1                2.5  
2                2.5  
3                2.5  
4                2.4  

输出值为:
0    0
1    2
2    2
3    2
4    2
Name: IrisType_num, dtype: int64


In [4]:
# 将输入和输出数据转换为张量，并创建一个TensorDataset
input = torch.Tensor(input.to_numpy())      # 创建类型为 torch.float32 的张量
print('\n输入格式: ', input.shape, input.dtype)     # 输入格式: torch.Size([150, 4]) torch.float32 
output = torch.tensor(output.to_numpy())        # 创建类型为 torch.int64 的张量
print('输出格式: ', output.shape, output.dtype)  # 输出格式: torch.Size([150]) torch.int64 
data = TensorDataset(input, output)    # 创建一个 torch.utils.data.TensorDataset 对象，以进一步操作数据


输入格式:  torch.Size([150, 4]) torch.float32
输出格式:  torch.Size([150]) torch.int64


In [5]:
# 使用 random_split 将数据集拆分为训练集、验证集和测试集
train_batch_size = 10        
number_rows = len(input)    # 数据集的大小或 Excel 表中的行数。
test_split = int(number_rows * 0.3)  
validate_split = int(number_rows * 0.2) 
train_split = number_rows - test_split - validate_split     
train_set, validate_set, test_set = random_split( 
    data, [train_split, validate_split, test_split])    
 
# 创建 DataLoader 以读取数据并按批次大小加载到内存中
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True) 
validate_loader = DataLoader(validate_set, batch_size=1) 
test_loader = DataLoader(test_set, batch_size=1)

In [6]:
# 定义模型参数
input_size = list(input.shape)[1]   # = 4。输入取决于我们最初向模型提供多少特征。在我们的情况下，每个预测值有 4 个特征。
learning_rate = 0.01 
output_size = len(labels)           # 输出是对三种鸢尾花进行预测的结果。

# 定义神经网络
class Network(nn.Module): 
   def __init__(self, input_size, output_size): 
       super(Network, self).__init__() 
        
       self.layer1 = nn.Linear(input_size, 24) 
       self.layer2 = nn.Linear(24, 24) 
       self.layer3 = nn.Linear(24, output_size) 

   def forward(self, x): 
       x1 = F.relu(self.layer1(x)) 
       x2 = F.relu(self.layer2(x1)) 
       x3 = self.layer3(x2) 
       return x3 
 
# 实例化模型
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
model = Network(input_size, output_size).to(device)

In [7]:
# 保存模型的函数
def saveModel(): 
    path = "./NetModel.pth" 
    torch.save(model.state_dict(), path)

In [8]:
# 使用分类交叉熵损失函数和 Adam 优化器定义损失函数和优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

In [9]:
# 训练函数
def train(num_epochs,device): 
    best_accuracy = 0.0 
     
    print("开始训练...") 
    for epoch in range(1, num_epochs+1): 
        running_train_loss = 0.0 
        running_accuracy = 0.0 
        running_val_loss = 0.0 
        total = 0 
 
        # 训练循环
        for data in train_loader: 
            inputs, outputs = data  # 获取输入和实际输出; data 是一个 [输入，输出] 的列表
            inputs, outputs = inputs.to(device), outputs.to(device)
            optimizer.zero_grad()   # 将参数梯度归零          
            predicted_outputs = model(inputs)   # 从模型中预测输出
            train_loss = loss_fn(predicted_outputs, outputs)   # 计算预测输出的损失
            train_loss.backward()   # 反向传播损失
            optimizer.step()        # 根据计算的梯度调整参数
            running_train_loss += train_loss.item()  # 跟踪损失值
 
        # 计算训练损失值
        train_loss_value = running_train_loss / len(train_loader) 
 
        # 验证循环
        with torch.no_grad(): 
            model.eval() 
            for data in validate_loader: 
                inputs, outputs = data 
                inputs, outputs = inputs.to(device), outputs.to(device)
                predicted_outputs = model(inputs) 
                val_loss = loss_fn(predicted_outputs, outputs) 
             
                # 预测输出中值最大的标签将是我们的预测
                _, predicted = torch.max(predicted_outputs, 1) 
                running_val_loss += val_loss.item()  
                total += outputs.size(0) 
                running_accuracy += (predicted == outputs).sum().item() 
 
        # 计算验证损失值
        val_loss_value = running_val_loss / len(validate_loader) 
                
        # 计算准确率，即验证批次中的正确预测数除以总预测数。
        accuracy = (100 * running_accuracy / total)     
 
        # 如果准确率是最佳的，则保存模型
        if accuracy > best_accuracy: 
            saveModel() 
            best_accuracy = accuracy 
         
        # 打印每个 epoch 的统计信息
        print('完成训练批次', epoch, '训练损失为: %.4f' % train_loss_value, '验证损失为: %.4f' % val_loss_value, '准确率为 %d %%' % (accuracy))

In [10]:
# 测试模型的函数
def test(): 
    # 加载我们在训练循环结束时保存的模型
    model = Network(input_size, output_size) 
    path = "NetModel.pth" 
    model.load_state_dict(torch.load(path)) 
     
    running_accuracy = 0 
    total = 0 
 
    with torch.no_grad(): 
        for data in test_loader: 
            inputs, outputs = data 
            outputs = outputs.to(torch.float32) 
            predicted_outputs = model(inputs) 
            _, predicted = torch.max(predicted_outputs, 1) 
            total += outputs.size(0) 
            running_accuracy += (predicted == outputs).sum().item() 
 
        print('基于', test_split, '个输入的测试集的模型准确率为: %d %%' % (100 * running_accuracy / total))    
 
 
# 可选: 测试哪种鸢尾花更容易预测
def test_species(): 
    # 加载我们在训练循环结束时保存的模型
    model = Network(input_size, output_size) 
    path = "NetModel.pth" 
    model.load_state_dict(torch.load(path)) 
     
    labels_length = len(labels) # 我们有多少种鸢尾花标签。在我们的数据库中为 3。
    labels_correct = list(0. for i in range(labels_length)) # 用于计算正确标签的列表 [多少正确的山鸢尾，多少正确的变色鸢尾，多少正确的维吉尼亚鸢尾] 
    labels_total = list(0. for i in range(labels_length))   # 用于保持每种类型的总标签数的列表 [总山鸢尾，总变色鸢尾，总维吉尼亚鸢尾] 
  
    with torch.no_grad(): 
        for data in test_loader: 
            inputs, outputs = data 
            predicted_outputs = model(inputs) 
            _, predicted = torch.max(predicted_outputs, 1) 
             
            label_correct_running = (predicted == outputs).squeeze() 
            label = outputs[0] 
            if label_correct_running.item():  
                labels_correct[label] += 1 
            labels_total[label] += 1  
  
    label_list = list(labels.keys()) 
    for i in range(output_size): 
        print('预测 %5s 的准确率为: %2d %%' % (label_list[i], 100 * labels_correct[i] / labels_total[i]))

In [11]:
if __name__ == "__main__": 
    num_epochs = 10
    train(num_epochs,device) 
    print('训练完成\n') 
    test() 
    test_species()

开始训练...
完成训练批次 1 训练损失为: 1.1453 验证损失为: 1.1163 准确率为 66 %
完成训练批次 2 训练损失为: 1.0914 验证损失为: 1.0686 准确率为 60 %
完成训练批次 3 训练损失为: 1.0510 验证损失为: 1.0318 准确率为 46 %
完成训练批次 4 训练损失为: 1.0156 验证损失为: 0.9953 准确率为 63 %
完成训练批次 5 训练损失为: 0.9806 验证损失为: 0.9562 准确率为 70 %
完成训练批次 6 训练损失为: 0.9433 验证损失为: 0.9119 准确率为 70 %
完成训练批次 7 训练损失为: 0.8803 验证损失为: 0.8640 准确率为 70 %
完成训练批次 8 训练损失为: 0.8390 验证损失为: 0.8164 准确率为 70 %
完成训练批次 9 训练损失为: 0.7877 验证损失为: 0.7673 准确率为 70 %
完成训练批次 10 训练损失为: 0.7362 验证损失为: 0.7183 准确率为 70 %
训练完成

基于 45 个输入的测试集的模型准确率为: 62 %
预测 Iris-setosa 的准确率为: 100 %
预测 Iris-versicolor 的准确率为:  0 %
预测 Iris-virginica 的准确率为: 100 %
